#Packages for Drive Files

In [2]:
# import packages
## for mount drive purpose
import os
from google.colab import drive

#Mount Drive

In [3]:
# mount drive
drive.mount('/content/drive/', force_remount=True)
os.chdir('/content/drive/My Drive/Colab_Notebooks/Graph_ML/semi_dr.shen/pecanpy_test')

Mounted at /content/drive/


# import ipynb packages

In [4]:
!pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=cd74f840f3ded103996061919905fa4ecf54b63b29a6d798758563c84a23dff6
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [15]:
import import_ipynb
from test_cases import Model, Case

#pecanpy

## check the pecanpy package

https://towardsdatascience.com/run-node2vec-faster-with-less-memory-using-pecanpy-1bdf31f136de

In [ ]:
!pip install pecanpy

     |████████████████████████████████| 24.1 MB 2.8 MB/s 
     |████████████████████████████████| 3.3 MB 32.3 MB/s 
     |████████████████████████████████| 34.5 MB 12 kB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
!pecanpy --input karate.edg --output karate.emb --mode SparseOTF

Took 00:00:00.45 to load Graph
Took 00:00:00.00 to pre-compute transition probabilities
Took 00:00:05.90 to generate walks
Took 00:00:00.50 to train embeddings


## Get the embedding matrix with the Karate Club

1. Need to find there file provided in the git for karate:
https://github.com/krishnanlab/PecanPy/blob/master/demo/karate.edg
2. the pip installed is an old version, which requires 2 more positional arguments: 'n_ckpts' and 'pb_len'. Thus doesn't work for the demo code.
3. I downloaded the py files and paste the main code from thier git: https://github.com/krishnanlab/PecanPy/tree/master/src/pecanpy
4. STILL! THE RELATIVE IMPORT DOESN'T WORK IN COLAB, HAVE TO FIX MANY LEVELS OF IMPORTS!! 
5. IMPORT FROM FOLDER DOESN'T WORK IN COLAB EITHER. I need to move their py files from other folders to the current directory.
6. They recently updated the packages for dense and sparse graphs, but didn't change in the pecanpy code. Fixed by finding the right class names in other modules. 
7. The demo code is then modifed to run this fixed version
8. embed() function is modified to take the walks as its input

### install required packages

In [6]:
! pip install -r requirements.txt

     |████████████████████████████████| 24.1 MB 4.7 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
     |████████████████████████████████| 34.5 MB 2.9 MB/s 
     |████████████████████████████████| 45 kB 2.3 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


### Main code(fixed version)

In [7]:
"""Different strategies for generating node2vec walks."""
import numpy as np
from gensim.models import Word2Vec
from numba import njit
from numba import prange
from numba.np.ufunc.parallel import _get_thread_id
from numba_progress import ProgressBar

from graph import BaseGraph
from dense_rw import DenseRWGraph
from sparse_rw import SparseRWGraph  
from typing_v2 import Embeddings
from typing_v2 import Float32Array
from typing_v2 import HasNbrs
from typing_v2 import List
from typing_v2 import MoveForward
from typing_v2 import Optional
from typing_v2 import Uint32Array
from typing_v2 import Uint64Array
from wrappers import Timer


class Base(BaseGraph):
    """Base node2vec object.
    This base object provides the skeleton for the node2vec walk algorithm,
    which consists of the ``simulate_walks`` method that generate node2vec
    random walks. In contrast to the original Python implementaion of node2vec,
    it is prallelized where each process generate walks independently.
    Note:
        The ``preprocess_transition_probs`` is required for implenetations that
        precomputes and store 2nd order transition probabilities.
    Examples:
        Generate node2vec embeddings
        >>> from pecanpy import node2vec
        >>>
        >>> # initialize node2vec object, similarly for SparseOTF and DenseOTF
        >>> g = node2vec.PreComp(p=0.5, q=1, workers=4, verbose=True)
        >>> # alternatively, can specify ``extend=True`` for using node2vec+
        >>>
        >>> # load graph from edgelist file
        >>> g.read_edg(path_to_edg_file, weighted=True, directed=False)
        >>> # precompute and save 2nd order transition probs (for PreComp only)
        >>> g.preprocess_transition_probs()
        >>>
        >>> # generate random walks, which could then be used to train w2v
        >>> walks = g.simulate_walks(num_walks=10, walk_length=80)
        >>>
        >>> # alternatively, generate the embeddings directly using ``embed``
        >>> emd = g.embed()
    """

    def __init__(
        self,
        p: float = 1,
        q: float = 1,
        workers: int = 1,
        verbose: bool = False,
        extend: bool = False,
        gamma: float = 0,
        random_state: Optional[int] = None,
    ):
        """Initializ node2vec base class.
        Args:
            p (float): return parameter, value less than 1 encourages returning
                back to previous vertex, and discourage for value grater than 1
                (default: 1).
            q (float): in-out parameter, value less than 1 encourages walks to
                go "outward", and value greater than 1 encourage walking within
                a localized neighborhood (default: 1)
            workers (int): number of threads to be spawned for runing node2vec
                including walk generation and word2vec embedding (default: 1)
            verbose (bool): show progress bar for walk generation.
            extend (bool): use node2vec+ extension if set to :obj:`True`
                (default: :obj:`False`).
            gamma (float): Multiplication factor for the std term of edge
                weights added to the average edge weights as the noisy edge
                threashold, only used by node2vec+ (default: 0)
            random_state (int, optional): Random seed for generating random
                walks. Note that to fully ensure reproducibility, use single
                thread (i.e., workers=1), and potentially need to set the
                Python environment variable ``PYTHONHASHSEED`` to match the
                random_state (default: :obj:`None`).
        """
        super().__init__()
        self.p = p
        self.q = q
        self.workers = workers  # TODO: not doing anything, need to fix.
        self.verbose = verbose
        self.extend = extend
        self.gamma = gamma
        self.random_state = random_state
        self._preprocessed: bool = False

    def _map_walk(self, walk_idx_ary: Uint32Array) -> List[str]:
        """Map walk from node index to node ID.
        Note:
            The last element in the ``walk_idx_ary`` encodes the effective walk
            length. Only walk indices up to the effective walk length are
            translated (mapped to node IDs).
        """
        end_idx = walk_idx_ary[-1]
        walk = [self.nodes[i] for i in walk_idx_ary[:end_idx]]
        return walk

    def simulate_walks(
        self,
        num_walks: int,
        walk_length: int,
    ) -> List[List[str]]:
        """Generate walks starting from each nodes ``num_walks`` time.
        Note:
            This is the master process that spawns worker processes, where the
            worker function ``node2vec_walks`` genearte a single random walk
            starting from a vertex of the graph.
        Args:
            num_walks (int): number of walks starting from each node.
            walks_length (int): length of walk.
        """
        self._preprocess_transition_probs()

        nodes = np.array(range(self.num_nodes), dtype=np.uint32)
        start_node_idx_ary = np.concatenate([nodes] * num_walks)
        tot_num_jobs = start_node_idx_ary.size

        random_state = self.random_state
        np.random.seed(random_state)
        np.random.shuffle(start_node_idx_ary)  # for balanced work load

        move_forward = self.get_move_forward()
        has_nbrs = self.get_has_nbrs()
        verbose = self.verbose

        # Acquire numba progress proxy for displaying the progress bar
        with ProgressBar(total=tot_num_jobs, disable=not verbose) as progress:
            walk_idx_mat = self._random_walks(
                tot_num_jobs,
                walk_length,
                random_state,
                start_node_idx_ary,
                has_nbrs,
                move_forward,
                progress,
            )

        # Map node index back to node ID
        walks = [self._map_walk(walk_idx_ary) for walk_idx_ary in walk_idx_mat]

        return walks

    @staticmethod
    @njit(parallel=True, nogil=True)
    def _random_walks(
        tot_num_jobs: int,
        walk_length: int,
        random_state: Optional[int],
        start_node_idx_ary: Uint32Array,
        has_nbrs: HasNbrs,
        move_forward: MoveForward,
        progress_proxy: ProgressBar,
    ) -> Uint32Array:
        """Simulate a random walk starting from start node."""
        # Seed the random number generator
        if random_state is not None:
            np.random.seed(random_state + _get_thread_id())

        # use the last entry of each walk index array to keep track of the
        # effective walk length
        walk_idx_mat = np.zeros((tot_num_jobs, walk_length + 2), dtype=np.uint32)
        walk_idx_mat[:, 0] = start_node_idx_ary  # initialize seeds
        walk_idx_mat[:, -1] = walk_length + 1  # set to full walk length by default

        for i in prange(tot_num_jobs):
            # initialize first step as normal random walk
            start_node_idx = walk_idx_mat[i, 0]
            if has_nbrs(start_node_idx):
                walk_idx_mat[i, 1] = move_forward(start_node_idx)
            else:
                walk_idx_mat[i, -1] = 1
                continue

            # start bias random walk
            for j in range(2, walk_length + 1):
                cur_idx = walk_idx_mat[i, j - 1]
                if has_nbrs(cur_idx):
                    prev_idx = walk_idx_mat[i, j - 2]
                    walk_idx_mat[i, j] = move_forward(cur_idx, prev_idx)
                else:
                    walk_idx_mat[i, -1] = j
                    break

            progress_proxy.update(1)

        return walk_idx_mat

    def setup_get_normalized_probs(self):
        """Transition probability computation setup.
        This is function performs necessary preprocessing of computing the
        average edge weights array, which is used later by the transition
        probability computation function ``get_extended_normalized_probs``,
        if node2vec+ is used. Otherwise, return the normal transition function
        ``get_noramlized_probs`` with a trivial placeholder for average edge
        weights array ``noise_thresholds``.
        """
        if self.extend:  # use n2v+
            get_normalized_probs = self.get_extended_normalized_probs
            noise_thresholds = self.get_noise_thresholds()
        else:  # use normal n2v
            get_normalized_probs = self.get_normalized_probs
            noise_thresholds = None
        return get_normalized_probs, noise_thresholds

    def preprocess_transition_probs(self):
        """Null default preprocess method."""
        pass

    def _preprocess_transition_probs(self):
        if not self._preprocessed:
            self.preprocess_transition_probs()
            self._preprocessed = True

    def embed(
        self,
        dim: int = 128,
        num_walks: int = 10,
        walk_length: int = 80,
        window_size: int = 10,
        epochs: int = 1,
        verbose: bool = False,
        walks: list = []
    ) -> Embeddings:
        """Generate embeddings.
        This is a shortcut function that combines ``simulate_walks`` with
        ``Word2Vec`` to generate the node2vec embedding.
        Note:
            The resulting embeddings are aligned with the graph, i.e., the
            index of embeddings is the same as that for the graph.
        Args:
            dim (int): dimension of the final embedding, default is 128
            num_walks (int): number of random walks generated using each node
                as the seed node, default is 10
            walk_length (int): length of the random walks, default is 80
            window_size (int): context window sized for training the
                ``Word2Vec`` model, default is 10
            epochs (int): number of epochs for training ``Word2Vec``, default
                is 1
            verbose (bool): print time usage for random walk generation and
                skip-gram training if set to True
        Return:
            Embeddings: The embedding matrix, each row is a node embedding
                vector. The index is the same as that for the graph.
        """
        # timed_walk = Timer("generate walks", verbose)(self.simulate_walks)
        timed_w2v = Timer("train embeddings", verbose)(Word2Vec)

        # walks = timed_walk(num_walks, walk_length)

        w2v = timed_w2v(
            walks,
            vector_size=dim,
            window=window_size,
            sg=1,
            min_count=0,
            workers=self.workers,
            epochs=epochs,
            seed=self.random_state,
        )


        return w2v.wv[self.nodes]


class FirstOrderUnweighted(Base, SparseRWGraph):
    """Directly sample edges for first order random walks."""

    def __init__(self, *args, **kwargs):
        """Initialize FirstOrderUnweighted mode."""
        Base.__init__(self, *args, **kwargs)

    def get_move_forward(self):
        """Wrap ``move_forward``."""
        indices = self.indices
        indptr = self.indptr

        @njit(nogil=True)
        def move_forward(cur_idx, prev_idx=None):
            start, end = indptr[cur_idx], indptr[cur_idx + 1]
            return indices[np.random.randint(start, end)]

        return move_forward


class PreCompFirstOrder(Base, SparseRWGraph):
    """Precompute transition probabilities for first order random walks."""

    def __init__(self, *args, **kwargs):
        """Initialize PreCompFirstOrder mode."""
        Base.__init__(self, *args, **kwargs)
        self.alias_j = self.alias_q = None

    def get_move_forward(self):
        """Wrap ``move_forward``."""
        indices = self.indices
        indptr = self.indptr

        alias_j = self.alias_j
        alias_q = self.alias_q

        @njit(nogil=True)
        def move_forward(cur_idx, prev_idx=None):
            start, end = indptr[cur_idx], indptr[cur_idx + 1]
            choice = alias_draw(alias_j[start:end], alias_q[start:end])

            return indices[indptr[cur_idx] + choice]

        return move_forward

    def preprocess_transition_probs(self):
        """Precompute and store first order transition probabilities."""
        data = self.data
        indices = self.indices
        indptr = self.indptr

        # Retrieve transition probability computation callback function
        get_normalized_probs = self.get_normalized_probs_first_order

        # Determine the dimensionality of the 1st order transition probs
        n_nodes = indptr.size - 1  # number of nodes
        n_probs = indptr[-1]  # total number of 1st order transition probs

        @njit(parallel=True, nogil=True)
        def compute_all_transition_probs():
            alias_j = np.zeros(n_probs, dtype=np.uint32)
            alias_q = np.zeros(n_probs, dtype=np.float32)

            for idx in range(n_nodes):
                start, end = indptr[idx], indptr[idx + 1]
                probs = get_normalized_probs(data, indices, indptr, idx)
                alias_j[start:end], alias_q[start:end] = alias_setup(probs)

            return alias_j, alias_q

        self.alias_j, self.alias_q = compute_all_transition_probs()


class PreComp(Base, SparseRWGraph):
    """Precompute transition probabilites.
    This implementation precomputes and store 2nd order transition probabilites
    first and uses read off transition probabilities during the process of
    random walk. The graph type used is ``SparseRWGraph``.
    Note:
        Need to call ``preprocess_transition_probs()`` first before generating
        walks.
    """

    def __init__(self, *args, **kwargs):
        """Initialize PreComp mode node2vec."""
        Base.__init__(self, *args, **kwargs)
        self.alias_dim: Optional[Uint32Array] = None
        self.alias_j: Optional[Uint32Array] = None
        self.alias_q: Optional[Float32Array] = None
        self.alias_indptr: Optional[Uint64Array] = None

    def get_move_forward(self):
        """Wrap ``move_forward``.
        This function returns a ``numba.njit`` compiled function that takes
        current vertex index (and the previous vertex index if available) and
        return the next vertex index by sampling from a discrete random
        distribution based on the transition probabilities that are read off
        the precomputed transition probabilities table.
        Note:
            The returned function is used by the ``simulate_walks`` method.
        """
        data = self.data
        indices = self.indices
        indptr = self.indptr
        p = self.p
        q = self.q
        get_normalized_probs = self.get_normalized_probs

        alias_j = self.alias_j
        alias_q = self.alias_q
        alias_indptr = self.alias_indptr
        alias_dim = self.alias_dim

        @njit(nogil=True)
        def move_forward(cur_idx, prev_idx=None):
            """Move to next node based on transition probabilities."""
            if prev_idx is None:
                normalized_probs = get_normalized_probs(
                    data,
                    indices,
                    indptr,
                    p,
                    q,
                    cur_idx,
                    None,
                    None,
                )
                cdf = np.cumsum(normalized_probs)
                choice = np.searchsorted(cdf, np.random.random())
            else:
                # Find index of neighbor (previous node) for reading alias
                start = indptr[cur_idx]
                end = indptr[cur_idx + 1]
                nbr_idx = np.searchsorted(indices[start:end], prev_idx)
                if indices[start + nbr_idx] != prev_idx:
                    print("FATAL ERROR! Neighbor not found.")

                dim = alias_dim[cur_idx]
                start = alias_indptr[cur_idx] + dim * nbr_idx
                end = start + dim
                choice = alias_draw(alias_j[start:end], alias_q[start:end])

            return indices[indptr[cur_idx] + choice]

        return move_forward

    def preprocess_transition_probs(self):
        """Precompute and store 2nd order transition probabilities.
        Each node contains n ** 2 number of 2nd order transition probabilities,
        where n is the number of neigbors of that specific nodes, since one can
        pick any one of its neighbors as the previous node and / or the next
        node. For each second order transition probability of a node, set up
        the alias draw table to be used during random walk.
        Note:
            Uses uint64 instaed of uint32 for tracking alias_indptr to prevent
            overflowing since the 2nd order transition probs grows much faster
            than the first order transition probs, which is the same as the
            total number of edges in the graph.
        """
        data = self.data
        indices = self.indices
        indptr = self.indptr
        p = self.p
        q = self.q

        # Retrieve transition probability computation callback function
        get_normalized_probs, noise_thresholds = self.setup_get_normalized_probs()

        # Determine the dimensionality of the 2nd order transition probs
        n_nodes = self.indptr.size - 1  # number of nodes
        n = self.indptr[1:] - self.indptr[:-1]  # number of nbrs per node
        n2 = np.power(n, 2)  # number of 2nd order trans probs per node

        # Set the dimensionality of alias probability table
        self.alias_dim = alias_dim = n
        self.alias_indptr = alias_indptr = np.zeros(self.indptr.size, dtype=np.uint64)
        alias_indptr[1:] = np.cumsum(n2)
        n_probs = alias_indptr[-1]  # total number of 2nd order transition probs

        @njit(parallel=True, nogil=True)
        def compute_all_transition_probs():
            alias_j = np.zeros(n_probs, dtype=np.uint32)
            alias_q = np.zeros(n_probs, dtype=np.float32)

            for idx in range(n_nodes):
                offset = alias_indptr[idx]
                dim = alias_dim[idx]

                nbrs = indices[indptr[idx] : indptr[idx + 1]]
                for nbr_idx in prange(n[idx]):
                    nbr = nbrs[nbr_idx]
                    probs = get_normalized_probs(
                        data,
                        indices,
                        indptr,
                        p,
                        q,
                        idx,
                        nbr,
                        noise_thresholds,
                    )

                    start = offset + dim * nbr_idx
                    end = start + dim
                    alias_j[start:end], alias_q[start:end] = alias_setup(probs)

            return alias_j, alias_q

        self.alias_j, self.alias_q = compute_all_transition_probs()


class SparseOTF(Base, SparseRWGraph):
    """Sparse graph transition on the fly.
    This implementation do *NOT* precompute transition probabilities in advance
    but instead calculate them on-the-fly during the process of random walk.
    The graph type used is ``SparseRWGraph``.
    """

    def __init__(self, *args, **kwargs):
        """Initialize PreComp mode node2vec."""
        Base.__init__(self, *args, **kwargs)

    def get_move_forward(self):
        """Wrap ``move_forward``.
        This function returns a ``numba.njit`` compiled function that takes
        current vertex index (and the previous vertex index if available) and
        return the next vertex index by sampling from a discrete random
        distribution based on the transition probabilities that are calculated
        on-the-fly.
        Note:
            The returned function is used by the ``simulate_walks`` method.
        """
        data = self.data
        indices = self.indices
        indptr = self.indptr
        p = self.p
        q = self.q

        get_normalized_probs, noise_thresholds = self.setup_get_normalized_probs()

        @njit(nogil=True)
        def move_forward(cur_idx, prev_idx=None):
            """Move to next node."""
            normalized_probs = get_normalized_probs(
                data,
                indices,
                indptr,
                p,
                q,
                cur_idx,
                prev_idx,
                noise_thresholds,
            )
            cdf = np.cumsum(normalized_probs)
            choice = np.searchsorted(cdf, np.random.random())

            return indices[indptr[cur_idx] + choice]

        return move_forward


class DenseOTF(Base, DenseRWGraph):
    """Dense graph transition on the fly.
    This implementation do *NOT* precompute transition probabilities in advance
    but instead calculate them on-the-fly during the process of random walk.
    The graph type used is ``DenseRWGraph``.
    """

    def __init__(self, *args, **kwargs):
        """Initialize DenseOTF mode node2vec."""
        Base.__init__(self, *args, **kwargs)

    def get_move_forward(self):
        """Wrap ``move_forward``.
        This function returns a ``numba.njit`` compiled function that takes
        current vertex index (and the previous vertex index if available) and
        return the next vertex index by sampling from a discrete random
        distribution based on the transition probabilities that are calculated
        on-the-fly.
        Note:
            The returned function is used by the ``simulate_walks`` method.
        """
        data = self.data
        nonzero = self.nonzero
        p = self.p
        q = self.q

        get_normalized_probs, noise_thresholds = self.setup_get_normalized_probs()

        @njit(nogil=True)
        def move_forward(cur_idx, prev_idx=None):
            """Move to next node."""
            normalized_probs = get_normalized_probs(
                data,
                nonzero,
                p,
                q,
                cur_idx,
                prev_idx,
                noise_thresholds,
            )
            cdf = np.cumsum(normalized_probs)
            choice = np.searchsorted(cdf, np.random.random())
            nbrs = np.where(nonzero[cur_idx])[0]

            return nbrs[choice]

        return move_forward


@njit(nogil=True)
def alias_setup(probs):
    """Construct alias lookup table.
    This code is modified from the blog post here:
    https://lips.cs.princeton.edu/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    , where you can find more details about how the method work. In general,
    the alias method improves the time complexity of sampling from a discrete
    random distribution to O(1) if the alias table is setup in advance.
    Args:
        probs (list(float32)): normalized transition probabilities array, could
            be in either list or NDArray, of float32 values.
    """
    k = probs.size
    q = np.zeros(k, dtype=np.float32)
    j = np.zeros(k, dtype=np.uint32)

    smaller = np.zeros(k, dtype=np.uint32)
    larger = np.zeros(k, dtype=np.uint32)
    smaller_ptr = 0
    larger_ptr = 0

    for kk in range(k):
        q[kk] = k * probs[kk]
        if q[kk] < 1.0:
            smaller[smaller_ptr] = kk
            smaller_ptr += 1
        else:
            larger[larger_ptr] = kk
            larger_ptr += 1

    while (smaller_ptr > 0) & (larger_ptr > 0):
        smaller_ptr -= 1
        small = smaller[smaller_ptr]
        larger_ptr -= 1
        large = larger[larger_ptr]

        j[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller[smaller_ptr] = large
            smaller_ptr += 1
        else:
            larger[larger_ptr] = large
            larger_ptr += 1

    return j, q


@njit(nogil=True)
def alias_draw(j, q):
    """Draw sample from a non-uniform discrete distribution using alias sampling."""
    k = j.size

    kk = np.random.randint(k)
    if np.random.rand() < q[kk]:
        return kk
    else:
        return j[kk]

# parameters

dim: int = 128,
num_walks: int = 10,
walk_length: int = 80,
window_size: int = 10,
epochs: int = 1,

# Modified demo code

In [ ]:
# load graph object using SparseOTF mode
g = SparseOTF(p=1, q=1, workers=1, verbose=False)
g.read_edg("karate.edg", weighted=False, directed=False)
# generate random walks
walks = g.simulate_walks(num_walks=10, walk_length=80)


### Compare their embedding function with n2v

The output is different.


In [ ]:
# use random walks to train embeddings
w2v_model = Word2Vec(walks, vector_size=128, window=10, min_count=0, sg=1, workers=1, epochs=1)

In [ ]:
Z1 = w2v_model.wv[g.nodes]

In [ ]:
print(g.nodes)

['1', '32', '22', '20', '18', '14', '13', '12', '11', '9', '8', '7', '6', '5', '4', '3', '2', '31', '10', '33', '29', '28', '17', '34', '15', '16', '19', '21', '23', '24', '30', '26', '25', '27']


In [ ]:
Z2 = g.embed(walks=walks,verbose=True)

Took 00:00:00.04 to train embeddings


In [ ]:
print(Z1[0])

[ 0.09955661 -0.06577659  0.15663469 -0.0226038  -0.06283473 -0.17676659
 -0.05379732  0.06729518  0.03701459  0.00145269  0.2450667  -0.08146148
 -0.07441966 -0.04600678 -0.01416261 -0.05349179  0.05099605  0.08768103
 -0.12734663  0.00587186  0.16730994  0.12447104  0.1025321   0.05059562
 -0.1718671   0.1185964  -0.17165709  0.0399924  -0.05845595 -0.05190227
 -0.26559263 -0.02096189 -0.00081739  0.01789241 -0.00880535  0.02066616
  0.21796896  0.00340983  0.21653605  0.02376861 -0.13036707  0.10410422
 -0.10283286 -0.09739631  0.03285079 -0.0163943   0.05351499 -0.06540594
 -0.09283355  0.09439462 -0.01490471 -0.02852398  0.1480384   0.1481084
  0.17119752  0.03900699  0.11885215 -0.03516997 -0.17469695  0.09332187
  0.09931957 -0.09200722  0.03358404  0.02454095  0.13857214 -0.08581842
  0.05714998  0.06754004 -0.05184027 -0.13891236  0.1172969  -0.01096353
 -0.12005869 -0.1239064  -0.01437692 -0.11847769  0.00751776 -0.00243801
 -0.09343819  0.11966772  0.00662899 -0.08735564  0.

In [ ]:
print(Z2[0])

[ 0.15609564  0.12784733 -0.11762779 -0.06594156 -0.03727759  0.01234312
 -0.07966905 -0.16150214 -0.09948079 -0.12302215 -0.14313099  0.11373156
  0.02245569  0.27968937 -0.00239474  0.06231022  0.05274847 -0.0200471
 -0.03437484  0.06686428  0.05516077  0.02273422 -0.11239681 -0.00047614
  0.09473525 -0.12359561  0.04675088 -0.12478731 -0.09149168  0.04554382
  0.1212319  -0.18324679  0.17859715 -0.14163205  0.22396383 -0.12850364
 -0.05614695 -0.01667082  0.1759377  -0.02552563  0.0732742  -0.05256329
 -0.02003324  0.03130965 -0.13734354 -0.02614268 -0.06387499  0.23307282
 -0.00879406  0.13069846  0.10325849 -0.04126237  0.01676075 -0.05981887
 -0.18377005  0.0772578  -0.00753567 -0.10629608  0.05139012  0.20332813
 -0.02126469  0.01938907  0.06573605 -0.06475727 -0.19479893  0.07140133
  0.06643978 -0.18150882 -0.16268492 -0.15328875  0.04364427 -0.08255892
 -0.11490618  0.06819894  0.10195261  0.19806972 -0.07278252  0.23141673
  0.11266776 -0.01231694 -0.06627136 -0.00215883  0.

###Check node order

**NOTE: the node order is what they collected from karate.edge. The order is not the same for Word2Vec embedding output**

In [ ]:
print(g.nodes)

['1', '32', '22', '20', '18', '14', '13', '12', '11', '9', '8', '7', '6', '5', '4', '3', '2', '31', '10', '33', '29', '28', '17', '34', '15', '16', '19', '21', '23', '24', '30', '26', '25', '27']


In [ ]:
print(type(g.nodes))

<class 'list'>


**The embedding matrix that they coded used the collected node order.**

In [ ]:
print(w2v_model.wv[g.nodes]) # from their emb function

[[ 0.09955661 -0.06577659  0.15663469 ... -0.17496707 -0.03551547
   0.05298061]
 [ 0.01663461 -0.05704665  0.14044228 ... -0.10534593  0.07712514
   0.03650736]
 [ 0.10221627 -0.06641143  0.16829978 ... -0.17229956 -0.02652555
   0.04165068]
 ...
 [ 0.0012593  -0.04182727  0.13827829 ... -0.09575876  0.09697586
   0.02158384]
 [ 0.00478785 -0.04838392  0.13143654 ... -0.1075692   0.08298232
   0.03370519]
 [ 0.01713896 -0.05889173  0.13738343 ... -0.11118698  0.07291349
   0.02111627]]


**The embedding output from wv**

In [ ]:
print(w2v_model.wv.vectors)

[[ 0.09955661 -0.06577659  0.15663469 ... -0.17496707 -0.03551547
   0.05298061]
 [ 0.02155339 -0.04537754  0.153262   ... -0.11868785  0.05721834
   0.02831656]
 [ 0.01602522 -0.05625723  0.13316976 ... -0.10076911  0.07284842
   0.02977898]
 ...
 [ 0.06025417 -0.05569373  0.15322822 ... -0.14789727  0.01464477
   0.04864639]
 [ 0.01713896 -0.05889173  0.13738343 ... -0.11118698  0.07291349
   0.02111627]
 [ 0.08395783 -0.06546038  0.16244197 ... -0.15888229 -0.00955479
   0.04725672]]


**The order of vectors is not the g.nodes order nor follows the node range (if 4 nodes, then node1, node2, node3, node4)**


In [ ]:
print(w2v_model.wv["32"] == w2v_model.wv.vectors[1])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]


In [ ]:
print(w2v_model.wv["1"] == w2v_model.wv.vectors[0])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]


In [ ]:
print(w2v_model.wv["2"] == w2v_model.wv.vectors[1])

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]


**Find out "output order" => "input node index"**


**To have keyed nodes are necessary**

In [ ]:
print("------Output embediing matrix row order => Input node order--------")
l = g.nodes.copy()
ori_len = range(len(l))
for i in ori_len:
  for node in g.nodes:
    result = (w2v_model.wv[node] ==  w2v_model.wv.vectors[i])
    if False not in result:
      print(i, "=>", node )
      l.remove(node)


------Output embediing matrix row order => Input node order--------
0 => 1
1 => 34
2 => 33
3 => 3
4 => 2
5 => 4
6 => 32
7 => 14
8 => 24
9 => 7
10 => 6
11 => 9
12 => 8
13 => 28
14 => 30
15 => 5
16 => 11
17 => 26
18 => 25
19 => 29
20 => 20
21 => 31
22 => 17
23 => 13
24 => 22
25 => 16
26 => 19
27 => 18
28 => 15
29 => 23
30 => 21
31 => 10
32 => 27
33 => 12


# Apply to test case 10

In [8]:
import networkx as nx
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import csv
import math

In [9]:
def save_edge(G, path):
  delimiter = "\t"
  with open(path, "w") as f:
    for (h, t) in G.edges:
        terms = (str(h), str(t))
        f.write(f"{delimiter.join(terms)}\n")

def save_np_to_csv(filename, array_to_save):
    np.savetxt(filename, array_to_save, delimiter=",")

def save_list_to_csv(filename,list_to_save):
  with open(filename, 'w') as f:        
      write = csv.writer(f)    
      write.writerow(list_to_save)

def split_sets(Y, test_ratio):
  t = test_ratio
  Y_1st_dim = Y.shape[0]

  np.random.seed(0)
  indices = np.random.permutation(Y_1st_dim)  #randomly permute the 1st indices

  # Generate indices for splits
  test_ind_split_point = math.floor(Y_1st_dim*t)
  test_idx, train_idx = indices[:test_ind_split_point], indices[test_ind_split_point:]
   
  return train_idx,  test_idx


## test 100 nodes first

In [16]:
n = 100
case = Case(n)
case_10 = case.case_10_fully_known()

In [ ]:
G = nx.from_numpy_matrix(case_10.X)

In [ ]:
# check node degree
for i in range(case_10.n):
  if G.degree[i] == 0:
    print(i)

In [ ]:
print(G.edges)

[(0, 27), (0, 31), (0, 39), (0, 48), (0, 51), (0, 52), (0, 55), (0, 56), (0, 57), (0, 73), (0, 89), (0, 90), (1, 6), (1, 27), (1, 60), (1, 64), (1, 73), (1, 89), (2, 4), (2, 6), (2, 8), (2, 13), (2, 26), (2, 31), (2, 37), (2, 38), (2, 40), (2, 49), (2, 62), (2, 72), (2, 77), (2, 82), (2, 90), (3, 7), (3, 8), (3, 20), (3, 35), (3, 38), (3, 49), (3, 60), (3, 74), (3, 78), (4, 6), (4, 11), (4, 29), (4, 40), (4, 44), (4, 56), (4, 68), (4, 94), (4, 95), (4, 97), (5, 27), (5, 35), (5, 43), (5, 52), (6, 9), (6, 26), (6, 27), (6, 28), (6, 32), (6, 35), (6, 48), (6, 55), (6, 57), (6, 83), (6, 88), (6, 90), (6, 91), (7, 31), (7, 45), (7, 47), (7, 51), (7, 59), (7, 66), (7, 81), (7, 83), (7, 85), (7, 91), (8, 13), (8, 22), (8, 26), (8, 29), (8, 30), (8, 34), (8, 38), (8, 40), (8, 62), (8, 66), (8, 91), (9, 10), (9, 32), (9, 36), (9, 39), (9, 74), (9, 84), (9, 87), (9, 94), (9, 95), (10, 16), (10, 25), (10, 29), (10, 52), (10, 71), (10, 79), (10, 83), (10, 93), (10, 94), (10, 96), (11, 38), (11, 4

In [ ]:
print(case_10.X[0][27])

1


In [ ]:
print(case_10.Y[0])

[1]


In [ ]:
save_edge(G, "output/case_10_100.edge")

In [10]:
g = SparseOTF(p=1, q=1, workers=1, verbose=False)
g.read_edg("output/case_10_100.edge", weighted=False, directed=False)
walks = g.simulate_walks(num_walks=10, walk_length=80)
Z = g.embed(walks=walks, window_size =2, verbose=True)

Took 00:00:00.12 to train embeddings


In [11]:
print(g.nodes)

['0', '27', '31', '39', '48', '51', '52', '55', '56', '57', '73', '89', '90', '1', '6', '60', '64', '2', '4', '8', '13', '26', '37', '38', '40', '49', '62', '72', '77', '82', '3', '7', '20', '35', '74', '78', '11', '29', '44', '68', '94', '95', '97', '5', '43', '9', '28', '32', '83', '88', '91', '45', '47', '59', '66', '81', '85', '22', '30', '34', '10', '36', '84', '87', '16', '25', '71', '79', '93', '96', '42', '61', '63', '76', '98', '12', '19', '46', '58', '86', '14', '18', '21', '69', '15', '41', '70', '54', '92', '17', '23', '67', '99', '24', '53', '50', '65', '75', '33', '80']


In [17]:
nodes = [str(i) for i in range(case_10.n)] # original order

In [18]:
print(nodes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']


In [19]:
test_idx = case_10.test_idx
train_idx = case_10.train_idx
print(test_idx)
print(train_idx)

[26 86  2 55 75 93 16 73 54 95 53 92 78 13  7 30 22 24 33  8]
[43 62  3 71 45 48  6 99 82 76 60 80 90 68 51 27 18 56 63 74  1 61 42 41
  4 15 17 40 38  5 91 59  0 34 28 50 11 35 23 52 10 31 66 57 79 85 32 84
 14 89 19 29 49 97 98 69 20 94 72 77 25 37 81 46 39 65 58 12 88 70 87 36
 21 83  9 96 67 64 47 44]


In [20]:
Y = case_10.Y_ori
Y_train = Y[train_idx].ravel()
Y_test = Y[test_idx].ravel()

In [21]:
z_train_idx = [g.nodes.index(str(i)) for i in train_idx]
z_test_idx = [g.nodes.index(str(i)) for i in test_idx]
print(z_train_idx)
print(z_test_idx)

[44, 26, 30, 66, 51, 4, 14, 92, 29, 73, 15, 99, 12, 39, 5, 1, 81, 8, 72, 34, 13, 71, 70, 85, 18, 84, 89, 24, 23, 43, 50, 53, 0, 59, 46, 95, 36, 33, 90, 6, 60, 2, 54, 9, 67, 56, 47, 62, 80, 11, 76, 37, 25, 42, 74, 83, 32, 40, 27, 28, 65, 22, 55, 77, 3, 96, 78, 75, 49, 86, 63, 61, 82, 48, 45, 69, 91, 16, 52, 38]
[21, 79, 17, 7, 97, 68, 64, 10, 87, 41, 94, 88, 35, 20, 31, 58, 57, 93, 98, 19]


In [22]:
Z_train = Z[z_train_idx]
Z_test = Z[z_test_idx]

In [23]:
print(Z_train)

[[ 0.01788941  0.13955449 -0.00275908 ... -0.07849458 -0.16136664
  -0.09861218]
 [ 0.00354991  0.12038469 -0.01453336 ... -0.0959447  -0.1621611
  -0.10813448]
 [ 0.00182336  0.11976834 -0.02268137 ... -0.08428904 -0.17093761
  -0.11315958]
 ...
 [ 0.01747253  0.12016302 -0.01743115 ... -0.08950182 -0.15185815
  -0.10277588]
 [-0.00460118  0.12037581 -0.01104969 ... -0.07866363 -0.14801843
  -0.10953969]
 [ 0.00737732  0.13305452 -0.02143178 ... -0.06990313 -0.16761956
  -0.09359503]]


In [24]:
clf = LinearDiscriminantAnalysis()
clf.fit(Z_train, Y_train)
acc = clf.score(Z_test, Y_test)
print(acc)


0.4


## output 3000 nodes embedding

In [ ]:
n = 3000
case = Case(n)
case_10 = case.case_10_fully_known()

In [ ]:
G = nx.from_numpy_matrix(case_10.X)

In [ ]:
save_edge(G, "output/case_10_3000.edge")

In [25]:
g = SparseOTF(p=1, q=1, workers=1, verbose=False)
g.read_edg("output/case_10_3000.edge", weighted=False, directed=False)
walks = g.simulate_walks(num_walks=10, walk_length=80)
Z = g.embed(walks=walks,window_size =2, verbose=True)

Took 00:00:10.38 to train embeddings


In [42]:
save_np_to_csv("output/case_10_3000_emb.csv", Z)

In [41]:
save_list_to_csv("output/case_10_3000_node_idx.csv", g.nodes)

In [28]:
test_idx = case_10.test_idx
train_idx = case_10.train_idx
Y = case_10.Y_ori
Y_train = Y[train_idx].ravel()
Y_test = Y[test_idx].ravel()
z_train_idx = [g.nodes.index(str(i)) for i in train_idx]
z_test_idx = [g.nodes.index(str(i)) for i in test_idx]
Z_train = Z[z_train_idx]
Z_test = Z[z_test_idx]
clf = LinearDiscriminantAnalysis()
clf.fit(Z_train, Y_train)
acc = clf.score(Z_test, Y_test)
print(acc)

0.4


#Load mat file

In [29]:
import scipy.io

In [30]:
mat_contents = scipy.io.loadmat('polblogs.mat')

In [31]:
sorted(mat_contents.keys())

['Adj',
 'AdjOri',
 'Edge',
 'K',
 'Vertex',
 'Y',
 'YOri',
 '__globals__',
 '__header__',
 '__version__']

In [ ]:
mat_contents['Adj']

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
mat_contents['Y']

array([[1],
       [1],
       [1],
       ...,
       [2],
       [2],
       [2]], dtype=uint8)

In [38]:
Y = mat_contents['Y']
print(Y.shape)

(1224, 1)


In [ ]:
A = mat_contents['Adj']
print(A.shape)

(1224, 1224)


In [ ]:
G = nx.from_numpy_matrix(A)

In [ ]:
save_edge(G, "output/mat.edge")

In [33]:
g = SparseOTF(p=1, q=1, workers=1, verbose=False)
g.read_edg("output/mat.edge", weighted=False, directed=False)
walks = g.simulate_walks(num_walks=10, walk_length=80)
Z = g.embed(walks=walks,window_size =2,verbose=True)

Took 00:00:03.20 to train embeddings


In [43]:
save_np_to_csv("output/mat_emb.csv", Z)

In [35]:
save_list_to_csv("output/mat_node_idx.csv", g.nodes)

In [36]:
train_idx, test_idx = split_sets(Y, 0.2)

In [39]:
Y_train = Y[train_idx].ravel()
Y_test = Y[test_idx].ravel()
z_train_idx = [g.nodes.index(str(i)) for i in train_idx]
z_test_idx = [g.nodes.index(str(i)) for i in test_idx]
Z_train = Z[z_train_idx]
Z_test = Z[z_test_idx]
clf = LinearDiscriminantAnalysis()
clf.fit(Z_train, Y_train)
acc = clf.score(Z_test, Y_test)
print(acc)

1.0
